<img src="kit_logo.jpg" width="272" height="125" align="right"/>

##### Lehrstuhl für Analytics and Statistics (IOR)
##### Prof. Dr. Oliver Grothe
##### SS 2022

# Multivariate Verfahren
## Übung 6

### Aufgabe 5

Betrachten Sie den Datensatz *`pisa.csv`* aus Ilias. Dieser enthält die Werte für
,,Lesekompetenz”, ,,Mathematische Grundbildung” und ,,Naturwissenschaftliche Grundbildung”
der Teilnehmerstaaten der PISA-Studie. 

Führen Sie eine Hauptkomponentenanalyse durch:

 a. Soll eine Hauptkomponentenanalyse eher mit der empirischen Kovarianzmatrix oder
mit der empirischen Korrelationsmatrix durchgeführt werden?

 b. Finden Sie die Hauptkomponenten des Modells und interpretieren Sie sie soweit möglich.

 c. Wie viele Komponenten benötigen Sie?

#### Datenvorbereitung

- Daten einlesen:

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("./pisa.csv", sep=',', header=None)
df = pd.DataFrame.to_numpy(df)
df[:10]


-  Plot der orginale Daten

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

fig = plt.figure(figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')

x1 = df[:, 0]
x2 = df[:, 1]
x3 = df[:, 2]
ax3d = ax.scatter3D(x1, x2, x3, c='r')
ax_vi = ax.view_init(None, 300)


- Berechnung der empirischen Kovarianzmatrix

In [ ]:
covar = np.cov(np.transpose(df))
covar


In [ ]:
from numpy import linalg as LA
w, v = LA.eig(covar)

print('Eigenvalues of covariance matrix: ', w)
print('Normalized eigenvectors of covariance matrix: ', '\n', v)
# sanity check sum of eigenvalues
print('Sum of eigenvalues: ', np.sum(w))
print('Trace of covariance matrix: ', np.trace(covar))


- Berechnung der empirischen Korrelationsmatrix

In [ ]:
# correlation coefficient matrix
cor = np.corrcoef(df.T)
cor


In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cor)

print('Eigenvalues of correlation coefficient matrix: ', eig_vals)
print('Normalized eigenvectors of correlation coefficient matrix: ', '\n', eig_vecs)
# sanity check sum of eigenvalues
print('Sum of eigenvalues: ', np.sum(eig_vals))
print('Trace of correlation coefficient matrix: ', np.trace(cor))


#### Hauptkomponentenanalyse

- Berechnung der zentrierten Daten

In [ ]:
import numpy.matlib
df_mean = np.matlib.repmat(np.mean(df, axis=0), 31, 1)
df2 = df-df_mean  # centered data
df2[:10]


- Model festlegen und anpassen

In [ ]:
# pca
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df)
coeff = pca.components_.T
score = pca.fit_transform(df)
latent = pca.explained_variance_
print('coeff: ', coeff)
print('score: ', score)
print('latent: ', latent)


- Plot der neue Koordinaten in gleichem euklidischem Raum

In [ ]:
fig = plt.figure(figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')
x1 = score[:, 0]
x2 = score[:, 1]
x3 = score[:, 2]
ax.scatter3D(x1, x2, x3, c='b', label='new coordinates from PCA')
y1 = df2[:, 0]
y2 = df2[:, 1]
y3 = df2[:, 2]
ax.scatter3D(y1, y2, y3, c='k', label='coordinates in euclidean space')
ax.view_init(None, 300)
ax.legend(loc='upper right')
plt.show()


- manuelle Berechnung der neue Koordinaten (Scores)

In [ ]:
score_manual = np.matmul(df2, coeff)
score_manual

# Exkurs

- Plot der Eigenvektoren (Loadings) 

In [ ]:
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)


In [ ]:
fig = plt.figure(figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')

y1 = df[:, 0]
y2 = df[:, 1]
y3 = df[:, 2]
ax.scatter3D(y1, y2, y3, c='k', label='coordinates in euclidean space')
for v in coeff.T:
    a = Arrow3D([np.mean(df, axis=0)[0], np.mean(df, axis=0)[0]+100*v[0]], [np.mean(df, axis=0)[1], np.mean(df, axis=0)[1]+100*v[1]],
                [np.mean(df, axis=0)[2], np.mean(df, axis=0)[2]+100*v[2]], mutation_scale=20,
                lw=1, arrowstyle="-|>", color="r")
    ax.add_artist(a)
ax.view_init(None, 300)
ax.legend(loc='upper right')
plt.draw()
plt.show()


-  Rekonstruierte Daten aus 2 Hauptkomponenten

In [ ]:
x_2 = np.matmul(score[:, :2], coeff[:, :2].T)


- Berechnung des Anteils der erklärten Varianz an der Gesamtvarianz 

In [ ]:
explained_variances = latent[:2].sum()
explained_variances_share = explained_variances/latent.sum()
print('explained variances of the frist 2 components: ', explained_variances_share)


* Prüfen, ob Abweichung zwischen Originaldaten und Rekonstruktion (mit  allen Komponenten!) in allen Zeilen in jeder (ursprünglichen) Dimension kleiner 0.01 ist

In [ ]:
x_rep = np.matmul(score, coeff.T)
(np.abs(df2-x_rep) < 0.0001).all()


- Plot der rekonstruierten Daten (2-dim) und Vergleich mit dem originalen Daten

In [ ]:
fig = plt.figure(figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')

x1 = x_2[:, 0]
x2 = x_2[:, 1]
x3 = x_2[:, 2]
ax.scatter3D(x1, x2, x3, c='b', label='reconstructed centered data')

y1 = df2[:, 0]
y2 = df2[:, 1]
y3 = df2[:, 2]
ax.scatter3D(y1, y2, y3, c='r', label='original centered data')
ax.view_init(None, 300)
ax.legend(loc='upper right')
plt.show()
